# Dipole Moment

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import csv
import datetime as dt
import re
import os
import json
import random
import time,pickle
import scipy.sparse as sp
from operator import itemgetter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

###### Read Graph and communities file

In [2]:
path= "C:/MyPath/Graphs/"

In [188]:
G = nx.read_weighted_edgelist(path + "retweet_graph_nationalkissingday_threshold_largest_CC.txt",
                              delimiter=",")

In [189]:
f1 = open("C:/MyPath/community1_nationalkissingday.txt" )
f2 = open("C:/MyPath/community2_nationalkissingday.txt" )

##### Compute DM metric

In [24]:
def convertIntoDirected(G):
    for edge in G.edges():
        node1 = edge[0];
        node2 = edge[1];
        G.add_edge(node2,node1);
    return G;

In [42]:
G = convertIntoDirected(G)
seed_nodes_num = int(len(G.nodes())*0.05); # seed nodes percenatage = 5 percent
seed_nodes_num = 1

In [43]:
lines1 = f1.readlines()
lines1= list(filter(('\n').__ne__, lines1 ))
left = []
dict_left = {}

for line in lines1:
    line = line.strip()
    left.append(line)
    dict_left[line] = 1
    
lines2 = f2.readlines()
lines2= list(filter(('\n').__ne__, lines2 ))
right = []
dict_right = {}

for line in lines2:
    line = line.strip()
    right.append(line)
    dict_right[line] = 1
   


In [44]:
def getNodesFromLabelsWithHighestDegree(G,k,flag): # first take the nodes with the highest degree according to the "flag" and then take the top $k$
    random_nodes = {}
    dict_degrees = {}
    for node in G.nodes():
        dict_degrees[node] = G.degree(node)
    sorted_dict = sorted(dict_degrees.items(), key=itemgetter(1), reverse=True) # sorts nodes by degrees
#sorted_dict = sorted_dict[:k];
    if(flag=="left"):
        count = 0
        for i in sorted_dict:
            if(count>k):
                break
            if(not dict_left.__contains__(i[0])):
                continue
            random_nodes[i[0]] = i[1]
            count += 1
    elif(flag=="right"):
        count = 0
        for i in sorted_dict:
            if(count>k):
                break
            if(not dict_right.__contains__(i[0])):
                continue
            random_nodes[i[0]] = i[1]
            count += 1
    else:
        count = 0
        for i in sorted_dict:
            if(count>k/2):
                break
            if(not dict_left.__contains__(i[0])):
                continue
            random_nodes[i[0]] = i[1]
            count += 1
        count = 0
        for i in sorted_dict:
            if(count>k/2):
                break
            if(not dict_right.__contains__(i[0])):
                continue
            random_nodes[i[0]] = i[1]
            count += 1

    return random_nodes;


In [45]:
left_seed_nodes = getNodesFromLabelsWithHighestDegree(G,seed_nodes_num,'left')
right_seed_nodes = getNodesFromLabelsWithHighestDegree(G,seed_nodes_num,'right')

In [47]:
G1 = nx.DiGraph()

dict_ids = {}
dict_ideos = {}

count = 0
for node in G.nodes():
    dict_ids[node] = count
    count += 1
    if(left_seed_nodes.__contains__(node)):
        dict_ideos[node] = 1
    elif(right_seed_nodes.__contains__(node)):
        dict_ideos[node] = -1
    else:
        dict_ideos[node] = 0

for node in G.nodes():
    node_id = dict_ids[node]
    node_ideo = dict_ideos[node]
    G.add_node(node, ideo=node_ideo)
    
G1= nx.relabel_nodes(G, dict_ids, copy=True)


In [31]:
savepath= "C:/MyPath/MBLB_graphs/"

In [48]:
nx.write_gml(G1, savepath+ 'politics_reddit' + '_mblbgraph.gml')

In [33]:
def Model(G,corenode,tol=10**-5,save_xi=True):
    #G: Graph to calculate opinions. The nodes have an attribute "ideo" with their ideology, set to 0 for all listeners, 1 and -1 for the elite.
    #corenode: Nodes that belong to the seed (Identifiers from the Graph G)
    #tol is the threshold for convergence. It will evaluate the difference between opinions at time t and t+1
    #save_xi: boolean to save results
    N=len(G.nodes())
    #print N

    #Get de adjacency matrix
    Aij = nx.adjacency_matrix(G)
    #Aij = sp.lil_matrix((N,N))
   #print Aij.shape
    #for o,d in G.edges():
    #    Aij[o,d]=1

    #Build the vectors with users opinions
    macro_v_current=[]
    v_current= []
    v_new = []
    dict_nodes = {}
    
    for nodo in G.nodes():
        dict_nodes[G.nodes[nodo]['ideo']] = G.nodes[nodo]['ideo'] #first ideo was label
        v_current.append(G.nodes[nodo]['ideo'])
        v_new.append(0.0)

    v_current = 1.*np.array(v_current)
    #f2 = open("analyze_venezuela/results_iter_0","wb");
    #pickle.dump(dict_nodes,f2);
    #f2.close();
    v_new = 1.*np.array(v_new)
    
    notconverged=len(v_current)
    times = 0;
    
    #Do as many times as required for convergence
    while notconverged > 0:
        times=times+1
        t=time.time()
        
        #for all nodes apart from corenode, calculate opinion as average of neighbors
        for j in np.setdiff1d(range(len(v_current)),corenode):
            nodosin=Aij.getrow(j).nonzero()[1]
            if len(nodosin) > 0:
                v_new[j]= np.mean(v_current[nodosin])
            else:
                v_new[j]=v_current[j]
                #nodos_changed[j]=nodos_changed[j] or (v_new[j]!=v_current[j])
       
        #update opinion
        for j in corenode:
            v_new[j]=v_current[j]

        diff=np.abs(v_current-v_new)
        notconverged=len(diff[diff>tol])
        v_current=v_new.copy()
        zz = 0
       #for nodo in G.nodes():
            #dict_nodes[G.node[nodo]['label']] = v_current[zz];
             #zz += 1;
       #f1=open("analyze_venezuela/results_iter_" + str(times),'wb')
       #print v_current;
       #pickle.dump(dict_nodes,f1)
       #f1.close();

    return v_current   

In [34]:
def GetPolarizationIndex(ideos):
    #Input: Vector with individuals Xi
    #Output: Polarization index, Area Difference, Normalized Pole Distance
    D=[]#POLE DISTANCE
    AP=[]#POSSITIVE AREA
    AN=[]#NEGATIVE AREA
    cgp=[]#POSSITIVE GRAVITY CENTER
    cgn=[]#NEGATIVE GRAVITY CENTER
    
    ideos.sort()
    hist,edg=np.histogram(ideos,np.linspace(-1,1.1,50))
    edg=edg[:len(edg)-1]
    AP=sum(hist[edg>0])
    AN=sum(hist[edg<0])
    AP0=1.*AP/(AP+AN)
    AN0=1.*AN/(AP+AN)
    cgp=sum(hist[edg>0]*edg[edg>0])/sum(hist[edg>0])
    cgn=sum(hist[edg<0]*edg[edg<0])/sum(hist[edg<0])
    D=cgp-cgn
    p1= 0.5*D*(1.-1.*abs(AP0-AN0))#polarization index
    DA=1.*abs(AP0-AN0)/(AP0+AN0)#Areas Difference
    D=0.5*D#Normalized Pole Distance
    return p1,DA,D

In [49]:
G = nx.read_gml(savepath+ 'politics_reddit' + '_mblbgraph.gml');
ideos = nx.get_node_attributes(G,'ideo');

In [50]:
corenode = []

for key in ideos.keys():
    if(ideos[key]==1 or ideos[key]==-1):
        corenode.append(key)
corenode= list(map(int, corenode))

In [51]:
v_current = Model(G,corenode)

In [53]:
#polarization index, Areas Difference, Normalized Pole Distance
GetPolarizationIndex(v_current)

(0.004862825825466136, 0.9941626470859319, 0.8330532515426113)